<a href="https://colab.research.google.com/github/annasajkh/GPT-2-Music/blob/main/GPT_2_Music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title run this to setup everything
#@markdown only runs this once
#converter from https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

%tensorflow_version 1.x
!pip install -q gpt-2-simple

from os import path
import gpt_2_simple as gpt2
import tensorflow as tf
from google.colab import files

from google.colab import files
from music21 import converter, instrument, note, chord, stream
import os

def notes2midi(notes_str):
  prediction_output = notes_str.replace("<|startoftext|>","").strip().split(",")

  offset = 0
  output_notes = []

  for pattern in prediction_output:
    try:
      if ("." in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split(".")
        notes = []

        for current_note in notes_in_chord:
          new_note = note.Note(int(current_note))
          new_note.storedInstrument = instrument.Piano()
          notes.append(new_note)

        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
      else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    except Exception as e:
      print("WARNING THE AI GENERATE INVALID NOTE")
    
    offset += 0.5 

  midi_stream = stream.Stream(output_notes)
  midi_stream.write("midi", fp= "generated_music.mid")

if not path.exists("checkpoint"):
  !gdown --id 1W6-ic9WQF83ozJor9ibF2ohzZ1njqP6_
  !tar -xf checkpoint_run1.tar

if not path.exists("models"):
  gpt2.download_gpt2(model_name="355M")

tf.reset_default_graph()
session = gpt2.start_tf_sess()
gpt2.load_gpt2(session)

In [ ]:
#@title run this to generate music
#@markdown the generated music is a midi file you can convert it to mp3 using online converter
print("generating music this will take a while...")
notes = gpt2.generate(
              session,
              temperature=0.9,
              top_k=40,
              batch_size=1,
              prefix="<|startoftext|>",
              truncate="<|endoftext|>",
              include_prefix=False,
              length=1024,
              return_as_list=True
              )[0]


notes2midi(notes)

files.download("generated_music.mid") 